In [ ]:
# This Notebook is used in conjunction with Email Notifier.ipynb as an Add-on Tool in Seeq Workbench.
# See https://seeq.atlassian.net/wiki/spaces/SQ/pages/961675391/Add-on+Tools for installation details. 

In [ ]:
# These variables must be set according to the location and name of your Email Notifier notebook.
# Typical installation via the Install script will put all of the notebooks in the same location, 
# so this is assumed for a customization-free installation.
import re
p = re.compile(r'\/(notebooks|apps|addon)\/(.*)\/.+$')
notifier_notebook_url = p.sub(r'/\1/\2/Email%20Notifier.ipynb', jupyter_notebook_url)
unsubscriber_notebook_url = p.sub(r'/\1/\2/Email%20Unsubscriber.ipynb', jupyter_notebook_url)

In [ ]:
from IPython.display import HTML, display, clear_output
from bs4 import BeautifulSoup
import pandas as pd
import inspect
import ipywidgets as w
import ipyvuetify as v
import pytz
import os
import requests
import urllib.parse as urlparse

from seeq import spy
from seeq.sdk.rest import ApiException

class NotificationScheduler:
    
    v.theme.themes.light.success = '#007960'
    v.theme.themes.light.primary = '#007960'
    v.theme.themes.light.info = '#2a5c84'
    
    TEMPLATING_EXPLANATION = """
    <p style="line-height:1.5; margin-bottom:10px; font-size:14px">
      Enter your template HTML below. Note that not all HTML tags or CSS attributes
      will be rendered by email clients. You can validate your template by sending
      yourself test emails or using an online checker like 
      <a href="https://www.htmlemailcheck.com/check/" target="_blank">https://www.htmlemailcheck.com/check/</a>.
      Some advanced editors, e.g., VSCode, will also validate HTML.
    </p>
    <p style="line-height:1.5; margin-bottom:10px; font-size:14px">
      Within your template, you can also access variables specific to this notifier job.
      The fields Condition Name, Condition ID, Schedule, Lookback Interval, Topic Document URL,
      and Workbook ID are supported within the <code>job</code> context. You can insert these special
      variables into your email by referencing the <code>job</code> context, followed by the name of
      the name of the variable enclosed by double-quotes and square brackets.
    </p>
    <p style="line-height:1.5; margin-bottom:10px; font-size:14px">For example:</p>
    <p style="line-height:1.5; margin-bottom:10px; font-size:14px">
      <strong>Condition Name:</strong> {job["Condition Name"]}
      <br>
      <strong>Topic Document URL:</strong> {job["Topic Document URL"]}
    </p>
    <p style="line-height:1.5; margin-bottom:10px; font-size:14px">
      You can access information about the triggering capsule in a similar manner -
      for example, capsule start or end and custom capsule properties.
    </p>
    <p style="line-height:1.5; margin-bottom:10px; font-size:14px">
      <strong>Capsule Start:</strong> {capsule["Capsule Start"]}
      <br>
      <strong>Capsule Property (ex. "Batch ID"):</strong> {capsule["Batch ID"]}
    </p>
    <p style="line-height:1.5; margin-bottom:10px; font-size:14px">
      Any elements that have a class of "seeq-auto-update" may be modified by this Notebook upon scheduling.
      For example, the Topic Document URL field is optional, so if one is not provided, the paragraph element
      with class <code>"seeq-auto-update"</code> and id <code>"tdu"</code> will be removed from the default
      template.
    </p>
    """
    
    SUBJECT_TEMPLATE_DEFAULT = 'Capsule alert for Condition {job["Condition Name"]}'
    HTML_TEMPLATE_DEFAULT = """
        <html>
            <head></head>
            <body>
                <b>Capsule alert!</b>
                <p>A new capsule was identified in the Scheduled Email Sender with Capsule Start {capsule["Capsule Start"]}</p>
                <p>Capsule details:</p>
                <p><b>Start Time: </b>{capsule["Capsule Start"]}</p>
                <p><b>End Time: </b>{capsule["Capsule End"]}</p>
                <p class="seeq-auto-update" id="tdu">Topic Document: <a href='{job["Topic Document URL"]}'>link</a></p>
                <p>Regards,</p>
                <p>Seeq Data Lab&nbsp;<img src="cid:sdl"/></p>
                <a class="seeq-auto-update" id="usu" href='unsubscriber_notebook_url'>Unsubscribe</a>
            </body>
        </html>
    """.replace('unsubscriber_notebook_url', unsubscriber_notebook_url)
    TO_DEFAULT = spy.user.email
    SCHEDULE_DEFAULT = 'Every 6 hours'
    LOOKBACK_DEFAULT = '1.5'
    TIME_ZONE_DEFAULT = 'UTC'
    CC_DEFAULT = ''
    BCC_DEFAULT = ''
    TOPIC_DEFAULT = ''
    
    DEFAULT_JOB_PARAMETERS = {
        'Condition ID': '',
        'Condition Name': '',
        'Schedule': SCHEDULE_DEFAULT,
        'Time Zone': TIME_ZONE_DEFAULT,
        'Lookback Interval': LOOKBACK_DEFAULT,
        'To': TO_DEFAULT,
        'Cc': CC_DEFAULT,
        'Bcc': BCC_DEFAULT,
        'Topic Document URL': TOPIC_DEFAULT,
        'Subject Template': SUBJECT_TEMPLATE_DEFAULT,
        'Html Template': HTML_TEMPLATE_DEFAULT,
        'Workbook ID': '',
        'Scheduled': False,
        'Stopped': False,
        'Inception': ''
    }
    
    JOB_PARAMETERS = pd.Series(list(DEFAULT_JOB_PARAMETERS.keys()), dtype='string')
    
    # The character limit on job indices is too small to use guids - use a shortened guid instead.
    def short_id(self, guid):
        return guid[-12:]
    
    def get_source_worksheet_url(self):
        host = spy.client.host[:-4]
        return f'{host}/workbook/{self.workbook_id}/worksheet/{self.worksheet_id}'
    
    
    def check_email_configured(self):
        p = re.compile(r'\/(notebooks|apps|addon)\/(.*)$')
        notifier_notebook_contents_url = p.sub(r'/api/contents/\2', notifier_notebook_url)
        notifier_notebook = requests.get(notifier_notebook_contents_url, cookies = { 'sq-auth': spy.client.auth_token })
        default_email_text = """'Email Address': 'email.sender@mycompany.com'"""
        if not notifier_notebook.text.find(default_email_text) == -1:
            edit_notifier_url = notifier_notebook_url.replace('/addon/', '/notebooks/')
            error_html = v.Html(tag='p',
                                children=[
                                    'Configure SMTP settings in ',
                                    v.Html(tag='a', attributes={'href': edit_notifier_url, 'target': '_blank'}, children=['Notifier Notebook']
                                           , style_='color: #B1D7CF'),
                                    ' before using Scheduler'
                                ])
            self.on_error(error_html)
    
    def get_existing_jobs(self):
        try:
            pulled_jobs = spy.jobs.pull(datalab_notebook_url=notifier_notebook_url, label=self.workbook_id, all=True)
        except:
            pulled_jobs = None
        existing_jobs = pd.DataFrame(columns=self.DEFAULT_JOB_PARAMETERS.keys()) \
            if pulled_jobs is None or pulled_jobs.empty \
            else pulled_jobs
        if not existing_jobs.empty:
            existing_jobs['Scheduled'] = True
        return existing_jobs.astype({'Stopped': 'bool'})
    
    def get_conditions_from_worksheet(self):
        worksheet_items = None
        with self.output:
            worksheet_items = spy.search(self.get_source_worksheet_url(), quiet=True)
            clear_output()
        conditions_from_worksheet = None
        if worksheet_items.empty:
            conditions_from_worksheet = pd.DataFrame()
        else:
            conditions_from_worksheet = worksheet_items[worksheet_items['Type'].str.contains('Condition')]
            conditions_from_worksheet.index = [
                self.short_id(guid) for guid in conditions_from_worksheet['ID'].to_list()
            ]
        return conditions_from_worksheet
    
    def initialize_jobs(self):
        self.jobs = self.get_existing_jobs()
        for key, condition in self.get_conditions_from_worksheet().iterrows():
            if key in self.jobs.index:
                # Rename from worksheet in case the name in the pickled job DataFrame is stale
                self.jobs.loc[key, 'Condition Name'] = condition['Name']
            else:
                new_job = pd.Series(self.DEFAULT_JOB_PARAMETERS)
                new_job.name = self.short_id(condition['ID'])
                new_job['Condition ID'] = condition['ID']
                new_job['Condition Name'] = condition['Name']
                new_job['Stopped'] = False
                new_job['Scheduled'] = False
                self.jobs = pd.concat([self.jobs, new_job.to_frame().T], ignore_index=True)

    
    def displayable_conditions(self):
        unscheduled = [
            {'text': job['Condition Name'], 'value': index} for index, job 
             in self.jobs[self.jobs['Scheduled'] == False].iterrows()
        ]
        previously_scheduled =  [
            {'text': job['Condition Name'], 'value': index} for index, job
             in self.jobs[self.jobs['Scheduled'] == True].iterrows()]
        
        formatted_conditions = []
        if unscheduled:
            formatted_conditions += [{'header': 'From Worksheet or Unscheduled'}] + unscheduled
        if previously_scheduled:
            formatted_conditions += [{'header': 'Scheduled'}] + previously_scheduled

        return formatted_conditions
    
    def displayable_jobs(self):
        exclude_columns = [
            'Condition ID',
            'Topic Document URL',
            'Lookback Interval',
            'Subject Template',
            'Html Template',
            'Workbook ID',
            'Scheduled'
        ]
        return self.jobs[self.jobs['Scheduled'] == True].map(lambda x: float('NaN') if x=='' else x) \
            .drop(columns = exclude_columns).dropna(axis='columns', how='all')

    # In some cases, a template will need to be adjusted before sending it to the notifier.
    # For example, the Topic Document URL is an optional job parameter, so if a user doesn't
    # provide it, the HTML should not include the associated element.
    def soup_up_html_template(self):
        condition_id = self.jobs.loc[self.selected_condition]['Condition ID']
        soup = BeautifulSoup(self.html.v_model, 'html.parser')
        for sauce in soup.find_all(attrs={'class': 'seeq-auto-update', 'id': 'usu'}):
            sauce['href'] = f'{unsubscriber_notebook_url}?workbookId={self.workbook_id}&conditionId={condition_id}'
        if not self.topic.v_model.strip():
            for sauce in soup.find_all(attrs={'class': 'seeq-auto-update', 'id': 'tdu'}):
                sauce.replace_with('')            
        return soup.prettify()
    
    def set_conditions(self):
        displayable_conditions = self.displayable_conditions()
        displayable_selected_condition = next(filter(lambda row: not 'header' in row, displayable_conditions), None)
        self.selected_condition = displayable_selected_condition['value'] if displayable_selected_condition else None
        
        self.condition_select.v_model = displayable_selected_condition
        self.condition_select.items = displayable_conditions
    
    def set_data_table(self):
        displayable_jobs = self.displayable_jobs()
        headers = [{'text': header, 'value': header } for header in displayable_jobs.keys()]        
        self.data_table.headers = headers
        self.data_table.items = displayable_jobs.to_dict('records')
    
    def handle_selection_change(self, data):
        self.set_selected_job_from_models(data)
        self.selected_condition = data
        self.set_models_from_selected_job()                         
    
    def set_selected_job_from_models(self, data):
        if not self.selected_condition or self.selected_condition == data:
            return
        self.jobs.loc[self.selected_condition, 'To'] = self.to.v_model
        self.jobs.loc[self.selected_condition, 'Cc'] = self.cc.v_model
        self.jobs.loc[self.selected_condition, 'Bcc'] = self.bcc.v_model
        self.jobs.loc[self.selected_condition, 'Schedule'] = self.schedule.v_model
        self.jobs.loc[self.selected_condition, 'Topic Document URL'] = self.topic.v_model
        self.jobs.loc[self.selected_condition, 'Lookback Interval'] = self.lookback_period.v_model
        self.jobs.loc[self.selected_condition, 'Subject Template'] = self.subject.v_model
        self.jobs.loc[self.selected_condition, 'Html Template'] = self.html.v_model
        self.jobs.loc[self.selected_condition, 'Time Zone'] = self.time_zone.v_model        
    
    def set_models_from_selected_job(self):
        self.to.v_model = self.jobs.loc[self.selected_condition]['To']
        self.schedule.v_model = self.jobs.loc[self.selected_condition]['Schedule']
        self.topic.v_model = self.jobs.loc[self.selected_condition]['Topic Document URL']
        self.lookback_period.v_model = self.jobs.loc[self.selected_condition]['Lookback Interval']
        self.time_zone.v_model = self.jobs.loc[self.selected_condition]['Time Zone']
        self.cc.v_model = self.jobs.loc[self.selected_condition]['Cc']
        self.bcc.v_model = self.jobs.loc[self.selected_condition]['Bcc']
        self.subject.v_model = self.jobs.loc[self.selected_condition]['Subject Template']
        self.html.v_model = self.jobs.loc[self.selected_condition]['Html Template']
        self.unschedule_button.disabled = not self.jobs.loc[self.selected_condition]['Scheduled']
        self.stopped_job_warning.children = ['This job is not currently running.'] if self.jobs.loc[self.selected_condition]['Stopped'] else ['']     
    
    def set_validations(self):
        required_fields = ['to', 'schedule', 'subject', 'html']
    
        for field in required_fields:
            getattr(self, f'{field}').on_event('change', self.validate_required_field)
        self.lookback_period.on_event('change', self.validate_lookback_period)
    
    def valid_lookback_period(self):
        try:
            return float(self.lookback_period.v_model) > 0
        except Exception as ex:
            return False
    
    def validate_all(self):
        return self.to.v_model and self.schedule.v_model and self.subject.v_model and self.html.v_model and self.valid_lookback_period()
    
    def validate_required_field(self, widget, event, data):
        if widget.v_model:
            widget.rules= [] 
            if self.validate_all():
                self.schedule_button.disabled = False
        else:
            widget.rules = ['is required.']
            self.schedule_button.disabled = True        
    
    def validate_lookback_period(self, widget, event, data):
        if not widget.v_model:
            widget.rules = ['is required.']
            self.schedule_button.disabled = True 
            return
        
        if self.valid_lookback_period():
            widget.rules= []
            if self.validate_all():
                self.schedule_button.disabled = False
        else:
            widget.rules = ['must be a positive number.']
            self.schedule_button.disabled = True 
            
    def on_error(self, e):
        if type(e) is type(v.Html()):
            message = e
        else:
            error_text = str(e.message) if hasattr(e, 'message') else str(e)
            message = f'Something went wrong: {error_text}'

        self.snackbar.color = 'red darken-2'
        self.snackbar.children = [message, self.close_snackbar]
        self.snackbar.timeout = 0
        self.snackbar.v_model = True

    def on_success(self, message):
        self.snackbar.color = 'success'
        self.snackbar.children = [message, self.close_snackbar]
        self.snackbar.timeout = 5000
        self.snackbar.v_model = True
    
    def on_close_snackbar(self, widget, event, data):
        self.snackbar.v_model = False

    def on_condition_select(self, widget, event, data):
        data = 0 if data=={} else data
        self.handle_selection_change(data)
    
    def on_reset_templates(self, widget, event, data):
        if not self.selected_condition:
            return
        self.subject.v_model = self.SUBJECT_TEMPLATE_DEFAULT
        self.subject.rules= []
        self.html.v_model = self.HTML_TEMPLATE_DEFAULT
        self.html.rules= []
        if self.validate_all():
            self.schedule_button.disabled = False
    
    def on_schedule(self, widget, event, data):
        if not self.validate_all():
            self.snackbar.color = 'error'
            self.snackbar.children = ['Please fix the errors above.', self.close_snackbar]
            self.snackbar.v_model = True
            return
        
        self.schedule_button.loading = True        
        self.jobs.loc[self.selected_condition] = pd.Series(
            {
                'Condition ID': self.jobs.loc[self.selected_condition]['Condition ID'],
                'Condition Name': self.jobs.loc[self.selected_condition]['Condition Name'],
                'Schedule': self.schedule.v_model,
                'Lookback Interval': self.lookback_period.v_model,
                'Time Zone': self.time_zone.v_model,
                'To': self.to.v_model,
                'Cc': self.cc.v_model,
                'Bcc': self.bcc.v_model,
                'Topic Document URL': self.topic.v_model,
                'Subject Template': self.subject.v_model,
                'Html Template': self.soup_up_html_template(),
                'Workbook ID': self.workbook_id,
                'Stopped': False,
                'Scheduled': True,
                'Inception': self.jobs.loc[self.selected_condition]['Inception'] or pd.Timestamp.now('UTC').isoformat()
            },
            name = self.selected_condition
        )
        new_jobs = self.jobs[self.jobs['Scheduled'] == True].sort_index()  
        try:
            spy.jobs.push(
                new_jobs,
                datalab_notebook_url=notifier_notebook_url,
                label=self.workbook_id,
                quiet = True
            )
        except Exception as e:
            msg = str(e)
            if 'Features/DataLab/ScheduledNotebooks/Enabled'  in msg and 'Forbidden' in msg:
                self.on_error('ScheduledNotebooks are not enabled.  Contact an administrator to change th Features/DataLab/ScheduledNotebooks/Enabled setting.')
            elif 'Features/DataLab/ScheduledNotebooks/MinimumScheduleFrequency' in msg:
                self.on_error(f'The schedule {self.jobs.loc[self.selected_condition]["Schedule"]} is too frequent.'
                              f'Contact an administrator to change the Features/DataLab/ScheduledNotebooks/MinimumScheduleFrequency setting.')
            else:
                self.on_error(e)
            self.schedule_button.loading = False
            return

        self.condition_select.items=self.displayable_conditions()
        self.set_data_table()
        self.unschedule_button.disabled = False
        self.schedule_button.loading = False
        self.on_success('Scheduled!')
            
    def on_unschedule(self, widget, event, data):
        self.unschedule_button.loading = False
        self.jobs.loc[self.selected_condition, 'Scheduled'] = False
        remaining_jobs = self.jobs[self.jobs['Scheduled'] == True].sort_index()
        try:
            if remaining_jobs.empty:
                spy.jobs.unschedule(
                    datalab_notebook_url=notifier_notebook_url,
                    label=self.workbook_id,
                    quiet=True
                )
            else:
                status_df = spy.jobs.push(
                    remaining_jobs,
                    datalab_notebook_url=notifier_notebook_url,
                    label=self.workbook_id,
                    quiet=True
                )
        except Exception as e:
            self.on_error(e)
            self.unschedule_button.loading = False
            return

        self.set_conditions()
        self.set_data_table()
        # self.set_selected_condition()
        self.unschedule_button.loading = False
        self.unschedule_button.disabled = True
        self.on_success('Unscheduled!')
    
    def __init__(self, workbook_id, worksheet_id):
        self.workbook_id = workbook_id
        self.worksheet_id = worksheet_id
        self.output = w.Output()
        
        self.close_snackbar = v.Btn(color='white', icon=True, children=[v.Icon(children=['mdi-window-close'])])
        self.snackbar = v.Snackbar(v_model=False, app=True, shaped=True, children=[])
        
        
        self.check_email_configured()
        
        self.initialize_jobs()
               
        self.selected_condition = None
        self.condition_select = v.Select(v_model=[], items=[], style_='margin-right: 8px; margin-top:0; padding-top:0', no_data_text='No conditions available')
        self.set_conditions()
        self.condition = v.Container(class_='text--primary',
                                     style_='width: 100%; max-width: 100%; padding: 5px 25px;',
                                     children=[v.Row(children=[
                                         v.Html(tag='p', style_='margin-right: 8px;font-size: 17px;margin-top: 4px; color: #007960;', children=['Condition']),
                                         self.condition_select
                                     ])])
        
        self.scheduling_tab = v.Tab(children=['Scheduling'])
        self.to = v.Textarea(v_model = None, label='To', hint='Separate email addresses with commas', rows=1, auto_grow = True)
        self.schedule = v.TextField(v_model = None, label='Schedule')
        self.topic = v.Textarea(v_model = None, label='Topic Document URL (optional)', rows=1, auto_grow = True)
        self.time_zone = v.Select(v_model=None, items=pytz.all_timezones, label='Time Zone')
        self.lookback_period = v.TextField(v_model = None, label='Check for new capsule starts within the last N days:')
        self.cc = v.Textarea(v_model = None, label='Cc', hint='Separate email addresses with commas', rows=1, auto_grow = True)
        self.bcc = v.Textarea(v_model = None, label='Bcc', hint='Separate email addresses with commas', rows=1, auto_grow = True)
        self.advanced_content = v.ExpansionPanelContent(children=[self.time_zone, self.lookback_period, self.cc, self.bcc])
        self.advanced = v.ExpansionPanels(children=[v.ExpansionPanel(children=[v.ExpansionPanelHeader(children=['Advanced']), self.advanced_content])])
        self.scheduling_content = v.TabItem(children = [self.to, self.schedule, self.topic, self.advanced], style_='padding: 15px')
        
        self.templates_tab = v.Tab(children=['Templates'])
        self.instructions_content = v.ExpansionPanelContent(children=[w.HTML(self.TEMPLATING_EXPLANATION)])
        self.instructions = v.ExpansionPanels(children=[v.ExpansionPanel(children=[v.ExpansionPanelHeader(children=['Instructions'], style_='font-size:16px'), self.instructions_content])])
        self.subject = v.Textarea(v_model = None, label='Subject Template', rows=1, auto_grow = True)
        self.html = v.Textarea(v_model = None, label='HTML Template', auto_grow = True)
        self.reset_templates_button = v.Btn(children=['Reset'], class_='mx-2')
        self.reset_templates_button.on_event('click', self.on_reset_templates)
        self.template_tools = v.Toolbar(children=[v.Spacer(), self.reset_templates_button], style_='box-shadow: none')
        self.templates_content = v.TabItem(children = [self.instructions, self.subject, self.html, self.template_tools], style_='padding: 15px')
            
        self.jobs_tab = v.Tab(children=['Jobs'])
        self.data_table = v.DataTable(headers= [], items=[], v_model=[], item_key='Condition Name', class_='elevation-1', no_data_text='No notifications scheduled.')
        self.data_content = v.TabItem(children = [self.data_table], class_='pa4')
        
        if self.jobs.empty or self.selected_condition not in self.jobs.index:
            unschedule_button_disabled = True
        else:
            unschedule_button_disabled = bool(self.jobs.loc[self.selected_condition, 'Scheduled'])
        
        self.stopped_job_warning = v.Subheader(children=[])
        self.schedule_button = v.Btn(children=['Schedule'],
                                     color='success',
                                     class_='mx-2',
                                     disabled=self.selected_condition == None)        
        self.unschedule_button = v.Btn(children=['Unschedule'],
                                       class_='mx-2',
                                       disabled=unschedule_button_disabled)
        
        if self.selected_condition is not None:
            self.set_models_from_selected_job()
        self.reset_templates_button.disabled = self.selected_condition == None
        self.set_data_table()
        self.set_validations()
        
        self.condition_select.on_event('change', self.on_condition_select)
        self.schedule_button.on_event('click', self.on_schedule)
        self.unschedule_button.on_event('click', self.on_unschedule)
        self.close_snackbar.on_event('click', self.on_close_snackbar)
        
        self.app = v.App(id='notification-scheduler')        
        self.appBar = v.AppBar(
            color='white',
            dense=True,
            children=[v.ToolbarTitle(children=['Email Notification Scheduler'])])
        self.tabs = v.Tabs(children = [self.scheduling_tab, self.scheduling_content, self.templates_tab, self.templates_content, self.jobs_tab, self.data_content])
        self.footer = v.Toolbar(children=[v.Spacer(), self.stopped_job_warning, self.unschedule_button, self.schedule_button], style_='box-shadow: none')
        
    def run(self):
        display(HTML("<style> div.output_subarea { max-width:100% !important; } .mdi-checkbox-marked, .mdi-minus-box { color: #007960 !important; } .v-toolbar__content { padding: 0 !important } .v-snack {position: absolute !important;top: -300px;right: 0 !important; left: unset !important;} </style>"))
        self.app.children = [self.appBar, self.condition, v.Divider(), self.tabs, self.footer, self.snackbar]
        return self.app

url = jupyter_notebook_url
query_parameters = urlparse.parse_qs(urlparse.urlparse(url).query)
workbook_id = query_parameters['workbookId'][0] if 'workbookId' in query_parameters else None
worksheet_id = query_parameters['worksheetId'][0] if 'worksheetId' in query_parameters else None

if not workbook_id or not worksheet_id:
    raise Exception('Workbook and Worksheet IDs must be supplied as query parameters in the URL for this Notebook')
    
notification_scheduler = NotificationScheduler(workbook_id, worksheet_id)
notification_scheduler.run()